In [1]:
# def converter_hora(valor):
#     if pd.isna(valor):
#         return np.nan
#     if valor == 2400:
#         valor = 0
#     hora_str = f"{int(valor):04d}"
#     return datetime.time(int(hora_str[:2]), int(hora_str[2:]))
# def combinar_data_hora(data, hora):
#     if pd.isna(data) or pd.isna(hora):
#         return np.nan
#     return datetime.datetime.combine(data, hora)
# def criar_coluna_datetime(df, col):
#     lista = []
#     for _, row in df[['DATE', col]].iterrows():
#         date = row['DATE']
#         time_val = row[col]

#         if pd.isnull(time_val):
#             lista.append(np.nan)
#         elif float(time_val) == 2400:
#             date += datetime.timedelta(days=1)
#             time_val = datetime.time(0, 0)
#             lista.append(datetime.datetime.combine(date, time_val))
#         else:
#             time_val = converter_hora(time_val)
#             lista.append(datetime.datetime.combine(date, time_val))
#     return pd.Series(lista)
# # df['SCHEDULED_DEPARTURE'] = criar_coluna_datetime(df, 'SCHEDULED_DEPARTURE')
# # df['DEPARTURE_TIME'] = df['DEPARTURE_TIME'].apply(converter_hora)
# # df['SCHEDULED_ARRIVAL'] = df['SCHEDULED_ARRIVAL'].apply(converter_hora)
# # df['WHEELS_OFF'] = df['WHEELS_OFF'].apply(converter_hora)
# # df['ARRIVAL_TIME'] = df['ARRIVAL_TIME'].apply(converter_hora)
# # df[df['TAIL_NUMBER']== '7819A']
# # print(converter_hora(1930))
# df_sample = df[df['TAIL_NUMBER'] == '7819A']
# df_sample['SCHEDULED_DEPARTURE'] = df_sample['SCHEDULED_DEPARTURE'].apply(converter_hora)
# df_sample['DEPARTURE_TIME'] = df_sample['DEPARTURE_TIME'].apply(converter_hora)
# df_sample['SCHEDULED_ARRIVAL'] = df_sample['SCHEDULED_ARRIVAL'].apply(converter_hora)
# df_sample['WHEELS_OFF'] = df_sample['WHEELS_OFF'].apply(converter_hora)
# df_sample['WHEELS_ON'] = df_sample['WHEELS_ON'].apply(converter_hora)
# df_sample['ARRIVAL_TIME'] = df_sample['ARRIVAL_TIME'].apply(converter_hora)
# df_sample['SCHEDULE_TIME'] = df_sample['SCHEDULE_TIME'].apply(converter_hora)
# df_sample.columns

In [2]:
# Previsão de Atrasos e Cancelamentos de Voos: Uma Análise Comparativa de Algoritmos de Machine Learning
# https://www.sciencedirect.com/science/article/pii/S0969699719302303
# https://journals.sagepub.com/doi/full/10.1177/0361198120930014
# Prever se um voo será atrasado/cancelado ou não, Random Forest,KNN,SVM
# Acurácia, F1-Score, Matriz de Confusão
# usar tipo um box plot comparando os modelos e suas acuracios tipo slide 2 data science
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import datetime as datetime

**DAY_OF_WEEK**: Representa o dia da semana em que o voo ocorre, geralmente numerado de 0 (segunda-feira) a 6 (domingo).

**AIRLINE**: O código identificador da companhia aérea que opera o voo, por exemplo, "AA" para American Airlines.

**TAIL_NUMBER**: O número de cauda da aeronave, um identificador único atribuído a cada avião.

**ORIGIN_AIRPORT**: O código do aeroporto de origem, geralmente utilizando o código IATA do aeroporto (ex: "JFK" para o Aeroporto John F. Kennedy).

**DESTINATION_AIRPORT**: O código do aeroporto de destino, também no formato IATA.

**SCHEDULED_DEPARTURE**: A hora programada de partida do voo, considerando o horário de 24 horas.

**DEPARTURE_TIME**: O horário real de partida do voo, registrado após o despejo da aeronave.

**DEPARTURE_DELAY**: O atraso (em minutos) na partida do voo em relação ao horário programado.

**TAXI_OUT**: O tempo (em minutos) que o avião levou para sair do portão de embarque até alcançar a pista de decolagem.

**WHEELS_OFF**: O horário em que as rodas da aeronave saem da pista, marcando o início da decolagem.

**SCHEDULED_TIME**: O tempo programado para o voo, em minutos, do momento da decolagem até o pouso.

**ELAPSED_TIME**: O tempo real de voo, em minutos, do momento em que a aeronave decola até o momento em que aterrissa.

**AIR_TIME**: O tempo total de voo (em minutos), excluindo o tempo de taxiamento.

**DISTANCE**: A distância total percorrida durante o voo, geralmente em milhas.

**WHEELS_ON**: O horário em que as rodas da aeronave tocam a pista durante o pouso.

**TAXI_IN**: O tempo (em minutos) que o avião leva desde o pouso até o momento em que chega ao portão de desembarque.

**SCHEDULED_ARRIVAL**: O horário programado de chegada do voo, com base no horário estimado.

**ARRIVAL_TIME**: O horário real de chegada do voo.

**ARRIVAL_DELAY**: O atraso (em minutos) na chegada do voo, comparando o horário real de chegada com o programado.

**DIVERTED**: Indica se o voo foi desviado para outro aeroporto devido a condições imprevistas (valor binário: 1 para desviado, 0 para não desviado).

**AIR_SYSTEM_DELAY**: O atraso causado por problemas no sistema de controle de tráfego aéreo.

**SECURITY_DELAY**: O atraso devido a questões de segurança, como inspeções ou controles de segurança.

**AIRLINE_DELAY**: O atraso causado por falhas ou problemas internos da companhia aérea, como atraso em procedimentos de manutenção.

**LATE_AIRCRAFT_DELAY**: O atraso devido a problemas com o voo anterior da aeronave (a aeronave que estava atrasada de outra viagem).

**WEATHER_DELAY**: O atraso causado por condições climáticas adversas, como tempestades ou nevoeiro.

**DELAYED**: Indica se o voo foi atrasado em relação ao horário programado (valor binário: 1 para atrasado, 0 para não atrasado).

**DATE**: A data do voo, geralmente no formato YYYY-MM-DD.

**PREV_ARRIVAL_DELAY**: O atraso na chegada do voo anterior para a mesma aeronave (em minutos), que pode influenciar no atraso do voo atual.

**PREV_DELAYED**: Indica se o voo anterior da aeronave foi atrasado (valor binário: 1 para atrasado, 0 para não atrasado).

In [3]:
import kagglehub

path = kagglehub.dataset_download("usdot/flight-delays")

print("Path to dataset files:", path)


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Path to dataset files: /Users/gustavocoradin/.cache/kagglehub/datasets/usdot/flight-delays/versions/1


In [4]:
import pandas as pd
df = pd.read_csv(path + "/flights.csv")

/var/folders/28/kqrw_4t50j7_wd11ljb0mrmh0000gn/T/ipykernel_67370/286890719.py:2: DtypeWarning: Columns (7,8) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path + "/flights.csv")


In [5]:
df.columns

Index(['YEAR', 'MONTH', 'DAY', 'DAY_OF_WEEK', 'AIRLINE', 'FLIGHT_NUMBER',
       'TAIL_NUMBER', 'ORIGIN_AIRPORT', 'DESTINATION_AIRPORT',
       'SCHEDULED_DEPARTURE', 'DEPARTURE_TIME', 'DEPARTURE_DELAY', 'TAXI_OUT',
       'WHEELS_OFF', 'SCHEDULED_TIME', 'ELAPSED_TIME', 'AIR_TIME', 'DISTANCE',
       'WHEELS_ON', 'TAXI_IN', 'SCHEDULED_ARRIVAL', 'ARRIVAL_TIME',
       'ARRIVAL_DELAY', 'DIVERTED', 'CANCELLED', 'CANCELLATION_REASON',
       'AIR_SYSTEM_DELAY', 'SECURITY_DELAY', 'AIRLINE_DELAY',
       'LATE_AIRCRAFT_DELAY', 'WEATHER_DELAY'],
      dtype='object')

Analisando o dataset / data cleaning

In [6]:
print(df.isna().sum() * 100 / len(df))

YEAR                    0.000000
MONTH                   0.000000
DAY                     0.000000
DAY_OF_WEEK             0.000000
AIRLINE                 0.000000
FLIGHT_NUMBER           0.000000
TAIL_NUMBER             0.252978
ORIGIN_AIRPORT          0.000000
DESTINATION_AIRPORT     0.000000
SCHEDULED_DEPARTURE     0.000000
DEPARTURE_TIME          1.480526
DEPARTURE_DELAY         1.480526
TAXI_OUT                1.530259
WHEELS_OFF              1.530259
SCHEDULED_TIME          0.000103
ELAPSED_TIME            1.805629
AIR_TIME                1.805629
DISTANCE                0.000000
WHEELS_ON               1.589822
TAXI_IN                 1.589822
SCHEDULED_ARRIVAL       0.000000
ARRIVAL_TIME            1.589822
ARRIVAL_DELAY           1.805629
DIVERTED                0.000000
CANCELLED               0.000000
CANCELLATION_REASON    98.455357
AIR_SYSTEM_DELAY       81.724960
SECURITY_DELAY         81.724960
AIRLINE_DELAY          81.724960
LATE_AIRCRAFT_DELAY    81.724960
WEATHER_DE

In [7]:
# trocando NaN por 0 já que 0 aqui significa ausência de atraso
delay_reason_cols = ['AIR_SYSTEM_DELAY', 'SECURITY_DELAY', 'AIRLINE_DELAY', 'LATE_AIRCRAFT_DELAY', 'WEATHER_DELAY']
df[delay_reason_cols] = df[delay_reason_cols].fillna(0)
df.head()

,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,...,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY
0,2015,1,1,4,AS,98,N407AS,ANC,SEA,5,...,408.0,-22.0,0,0,NaN,0.0,0.0,0.0,0.0,0.0
1,2015,1,1,4,AA,2336,N3KUAA,LAX,PBI,10,...,741.0,-9.0,0,0,NaN,0.0,0.0,0.0,0.0,0.0
2,2015,1,1,4,US,840,N171US,SFO,CLT,20,...,811.0,5.0,0,0,NaN,0.0,0.0,0.0,0.0,0.0
3,2015,1,1,4,AA,258,N3HYAA,LAX,MIA,20,...,756.0,-9.0,0,0,NaN,0.0,0.0,0.0,0.0,0.0
4,2015,1,1,4,AS,135,N527AS,SEA,ANC,25,...,259.0,-21.0,0,0,NaN,0.0,0.0,0.0,0.0,0.0


In [8]:
df.drop(['CANCELLATION_REASON', 'FLIGHT_NUMBER', 'CANCELLED'], axis=1, inplace=True, errors='ignore')

# removendo TAIL_NUMBER devido a sua alta cardinalidade
df.drop('TAIL_NUMBER', axis=1, inplace=True)

# removendo ORIGIN_AIRPORT, DESTINATION_AIRPORT, AIRLINE pois apresentam pouco ganho de
# informação para o modelo e deixam o dataset muito esparso
df.drop(['ORIGIN_AIRPORT', 'DESTINATION_AIRPORT', 'AIRLINE'], axis=1, inplace=True, errors='ignore')
df = df.dropna(subset=['DEPARTURE_TIME','DEPARTURE_DELAY','TAXI_OUT','WHEELS_OFF','SCHEDULED_TIME','ELAPSED_TIME','AIR_TIME','WHEELS_ON','TAXI_IN','ARRIVAL_TIME','ARRIVAL_DELAY'])

In [9]:
print(df.isna().sum() * 100 / len(df))

YEAR                   0.0
MONTH                  0.0
DAY                    0.0
DAY_OF_WEEK            0.0
SCHEDULED_DEPARTURE    0.0
DEPARTURE_TIME         0.0
DEPARTURE_DELAY        0.0
TAXI_OUT               0.0
WHEELS_OFF             0.0
SCHEDULED_TIME         0.0
ELAPSED_TIME           0.0
AIR_TIME               0.0
DISTANCE               0.0
WHEELS_ON              0.0
TAXI_IN                0.0
SCHEDULED_ARRIVAL      0.0
ARRIVAL_TIME           0.0
ARRIVAL_DELAY          0.0
DIVERTED               0.0
AIR_SYSTEM_DELAY       0.0
SECURITY_DELAY         0.0
AIRLINE_DELAY          0.0
LATE_AIRCRAFT_DELAY    0.0
WEATHER_DELAY          0.0
dtype: float64


In [10]:
# criando uma coluna para ver se o voo atrasou ou não (0,1)
df['DELAYED'] = df['ARRIVAL_DELAY'].apply(lambda x: 1 if x > 15 else 0)

# REMOVENDO FEATURES COM VAZAMENTO TEMPORAL (informações disponíveis apenas após o voo)
temporal_leakage_cols = [
    'ARRIVAL_DELAY',    # Target leakage - é o que queremos prever
    'ARRIVAL_TIME',     # Só conhecido após o voo
    'WHEELS_ON',        # Só conhecido após o voo  
    'TAXI_IN',          # Só conhecido após o voo
    'DEPARTURE_TIME',   # Pode ser conhecido, mas vamos ser conservadores
    'DEPARTURE_DELAY',  # Pode causar vazamento
    'WHEELS_OFF',       # Só conhecido após decolagem
    'TAXI_OUT',         # Só conhecido após decolagem
    'ELAPSED_TIME',     # Só conhecido após o voo
    'AIR_TIME'          # Só conhecido após o voo
]

print(f"Removendo {len(temporal_leakage_cols)} colunas com vazamento temporal:")
print(temporal_leakage_cols)
df = df.drop(columns=temporal_leakage_cols, errors='ignore')

Removendo 10 colunas com vazamento temporal:
['ARRIVAL_DELAY', 'ARRIVAL_TIME', 'WHEELS_ON', 'TAXI_IN', 'DEPARTURE_TIME', 'DEPARTURE_DELAY', 'WHEELS_OFF', 'TAXI_OUT', 'ELAPSED_TIME', 'AIR_TIME']


In [11]:
delay = df[df['DELAYED'] == 1]
on_time = df[df['DELAYED'] == 0]

delay_count = delay.shape[0]
on_time_count = on_time.shape[0]

total = delay_count + on_time_count
delay_percentage = (delay_count / total) * 100
on_time_percentage = (on_time_count / total) * 100

print(f"Não atrasados: {on_time_count:,}".replace(",", "."))
print(f"Atrasados: {delay_count:,}".replace(",", "."))
print(f"Percentual de voos não atrasados: {on_time_percentage:.2f}%")
print(f"Percentual de voos atrasados: {delay_percentage:.2f}%")


Não atrasados: 4.690.510
Atrasados: 1.023.498
Percentual de voos não atrasados: 82.09%
Percentual de voos atrasados: 17.91%


In [12]:
from imblearn.under_sampling import RandomUnderSampler

X = df.drop(columns=['DELAYED'])
y = df['DELAYED']

rus = RandomUnderSampler(random_state=42)
X_resampled, y_resampled = rus.fit_resample(X, y)


df_undersampled = pd.concat([X_resampled, y_resampled], axis=1)

on_time_count = (df_undersampled['DELAYED'] == 0).sum()
delay_count = (df_undersampled['DELAYED'] == 1).sum()
total = on_time_count + delay_count


on_time_percentage = (on_time_count / total) * 100
delay_percentage = (delay_count / total) * 100

print(f"Não atrasados: {on_time_count:,}".replace(",", "."))
print(f"Atrasados: {delay_count:,}".replace(",", "."))
print(f"Percentual de voos não atrasados: {on_time_percentage:.2f}%")
print(f"Percentual de voos atrasados: {delay_percentage:.2f}%")

Não atrasados: 1.023.498
Atrasados: 1.023.498
Percentual de voos não atrasados: 50.00%
Percentual de voos atrasados: 50.00%


In [13]:
# criando uma coluna date com o dia,mês e ano convertidos
# df['DATE'] = pd.to_datetime(df[['YEAR','MONTH', 'DAY']])
# df.drop(['YEAR', 'MONTH', 'DAY'], axis=1, inplace=True)

In [14]:
df.head()

,YEAR,MONTH,DAY,DAY_OF_WEEK,SCHEDULED_DEPARTURE,SCHEDULED_TIME,DISTANCE,SCHEDULED_ARRIVAL,DIVERTED,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY,DELAYED
0,2015,1,1,4,5,205.0,1448,430,0,0.0,0.0,0.0,0.0,0.0,0
1,2015,1,1,4,10,280.0,2330,750,0,0.0,0.0,0.0,0.0,0.0,0
2,2015,1,1,4,20,286.0,2296,806,0,0.0,0.0,0.0,0.0,0.0,0
3,2015,1,1,4,20,285.0,2342,805,0,0.0,0.0,0.0,0.0,0.0,0
4,2015,1,1,4,25,235.0,1448,320,0,0.0,0.0,0.0,0.0,0.0,0


In [15]:
pd.set_option('display.max_columns', None)
df.head()

,YEAR,MONTH,DAY,DAY_OF_WEEK,SCHEDULED_DEPARTURE,SCHEDULED_TIME,DISTANCE,SCHEDULED_ARRIVAL,DIVERTED,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY,DELAYED
0,2015,1,1,4,5,205.0,1448,430,0,0.0,0.0,0.0,0.0,0.0,0
1,2015,1,1,4,10,280.0,2330,750,0,0.0,0.0,0.0,0.0,0.0,0
2,2015,1,1,4,20,286.0,2296,806,0,0.0,0.0,0.0,0.0,0.0,0
3,2015,1,1,4,20,285.0,2342,805,0,0.0,0.0,0.0,0.0,0.0,0
4,2015,1,1,4,25,235.0,1448,320,0,0.0,0.0,0.0,0.0,0.0,0


In [16]:
airlines_dict = {
    'UA': 'United Air Lines Inc.',
    'AA': 'American Airlines Inc.',
    'US': 'US Airways Inc.',
    'F9': 'Frontier Airlines Inc.',
    'B6': 'JetBlue Airways',
    'OO': 'Skywest Airlines Inc.',
    'AS': 'Alaska Airlines Inc.',
    'NK': 'Spirit Air Lines',
    'WN': 'Southwest Airlines Co.',
    'DL': 'Delta Air Lines Inc.',
    'EV': 'Atlantic Southeast Airlines',
    'HA': 'Hawaiian Airlines Inc.',
    'MQ': 'American Eagle Airlines Inc.',
    'VX': 'Virgin America'
}


In [17]:
pd.set_option('display.max_columns', None)
df.head(5)

,YEAR,MONTH,DAY,DAY_OF_WEEK,SCHEDULED_DEPARTURE,SCHEDULED_TIME,DISTANCE,SCHEDULED_ARRIVAL,DIVERTED,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY,DELAYED
0,2015,1,1,4,5,205.0,1448,430,0,0.0,0.0,0.0,0.0,0.0,0
1,2015,1,1,4,10,280.0,2330,750,0,0.0,0.0,0.0,0.0,0.0,0
2,2015,1,1,4,20,286.0,2296,806,0,0.0,0.0,0.0,0.0,0.0,0
3,2015,1,1,4,20,285.0,2342,805,0,0.0,0.0,0.0,0.0,0.0,0
4,2015,1,1,4,25,235.0,1448,320,0,0.0,0.0,0.0,0.0,0.0,0


# IMPLEMENTAÇÃO KNN

In [18]:
# Pegando samples de 100k
df_delayed = df[df['DELAYED'] == 1]
df_on_time = df[df['DELAYED'] == 0]

# Definir o tamanho da amostra para cada classe
sample_size_per_class = 50000

# Amostrar aleatoriamente 50k de cada classe
df_delayed_sample = df_delayed.sample(n=sample_size_per_class, random_state=42)
df_on_time_sample = df_on_time.sample(n=sample_size_per_class, random_state=42)

# Combinar as amostras
df_sample = pd.concat([df_delayed_sample, df_on_time_sample])

# Embaralhar o DataFrame resultante para misturar as classes
df_sample = df_sample.sample(frac=1, random_state=42).reset_index(drop=True)

print("Shape do DataFrame original:", df.shape)
print("Shape do DataFrame amostrado:", df_sample.shape)
print("\Distribuição da classe 'DELAYED' no DataFrame amostrado:")
print(df_sample['DELAYED'].value_counts())

Shape do DataFrame original: (5714008, 15)
Shape do DataFrame amostrado: (100000, 15)
\Distribuição da classe 'DELAYED' no DataFrame amostrado:
DELAYED
0    50000
1    50000
Name: count, dtype: int64


# Task
Aplicar um modelo KNN nos dados de voos, exibir as métricas de acurácia e um relatório detalhado de custo/desempenho computacional.

## Dividir os dados

### Subtask:
Separar o DataFrame amostrado (`df_sample`) em conjuntos de treino e teste para os recursos (X) e o alvo (y).


**Reasoning**:
Split the sampled DataFrame into training and testing sets for features and the target variable using train_test_split.



In [19]:
from sklearn.model_selection import train_test_split

X_sample = df_sample.drop(columns=['DELAYED'])
y_sample = df_sample['DELAYED']

# Hold-out 70% treino, 30% teste com amostragem estratificada
X_train, X_test, y_train, y_test = train_test_split(
    X_sample, y_sample,
    test_size=0.3,
    stratify=y_sample,
    random_state=42
)


In [20]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


## Treinar o modelo knn

### Subtask:
Treinar um modelo KNN usando os dados de treino.


**Reasoning**:
Train a KNN model using the training data.



In [21]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors=15)
knn.fit(X_train_scaled, y_train)


KNeighborsClassifier(n_neighbors=15)

## Fazer previsões

### Subtask:
Usar o modelo treinado para fazer previsões no conjunto de teste.


**Reasoning**:
Use the trained KNN model to make predictions on the test feature set.



In [22]:
y_pred = knn.predict(X_test_scaled)


## Avaliar a acurácia

### Subtask:
Calcular a acurácia do modelo comparando as previsões com os valores reais do conjunto de teste.


**Reasoning**:
Calculate the accuracy of the model by comparing the predicted values with the actual values from the test set.



In [23]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()

tpr = recall  # mesmo que tp / (tp + fn)
tnr = tn / (tn + fp)

cm = confusion_matrix(y_test, y_pred)

print("Acurácia:", accuracy)
print("Precisão:", precision)
print("Recall (TPR):", recall)
print("F1-score:", f1)
print("True Negative Rate (TNR):", tnr)
print("Matriz de Confusão:")
print(cm)


Acurácia: 0.9158333333333334
Precisão: 0.9964186231595702
Recall (TPR): 0.8346666666666667
F1-score: 0.9083983312171232
True Negative Rate (TNR): 0.997
Matriz de Confusão:
[[14955    45]
 [ 2480 12520]]


In [24]:
# treino
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

# Previsão no conjunto de treino (CORRIGIDO - usando dados escalonados)
y_train_pred = knn.predict(X_train_scaled)

# Métricas no treino
accuracy_train = accuracy_score(y_train, y_train_pred)
precision_train = precision_score(y_train, y_train_pred)
recall_train = recall_score(y_train, y_train_pred)
f1_train = f1_score(y_train, y_train_pred)
tn, fp, fn, tp = confusion_matrix(y_train, y_train_pred).ravel()
tpr_train = recall_train  # tp / (tp + fn)
tnr_train = tn / (tn + fp)

print("### Métricas no TREINO ###")
print("Acurácia:", accuracy_train)
print("Precisão:", precision_train)
print("Recall (TPR):", recall_train)
print("F1-score:", f1_train)
print("True Negative Rate (TNR):", tnr_train)

print("\n### Métricas no TESTE ###")
print("Acurácia:", accuracy)
print("Precisão:", precision)
print("Recall (TPR):", recall)
print("F1-score:", f1)
print("True Negative Rate (TNR):", tnr)


### Métricas no TREINO ###
Acurácia: 0.9244857142857142
Precisão: 0.9971889431764942
Recall (TPR): 0.8513714285714286
F1-score: 0.9185290219167104
True Negative Rate (TNR): 0.9976

### Métricas no TESTE ###
Acurácia: 0.9158333333333334
Precisão: 0.9964186231595702
Recall (TPR): 0.8346666666666667
F1-score: 0.9083983312171232
True Negative Rate (TNR): 0.997


In [25]:
# EXECUTANDO VALIDAÇÃO CRUZADA PARA DETECTAR OVERFITTING
print("\n" + "="*50)
print("VALIDAÇÃO CRUZADA 5-FOLDS")
print("="*50)

from sklearn.model_selection import cross_validate, StratifiedKFold
from sklearn.metrics import make_scorer
from sklearn.pipeline import Pipeline

# Criando pipeline com normalização + KNN
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('knn', KNeighborsClassifier(n_neighbors=15))
])

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

scoring = {
    'accuracy': make_scorer(accuracy_score),
    'precision': make_scorer(precision_score),
    'recall': make_scorer(recall_score),
    'f1': make_scorer(f1_score)
}

cv_results = cross_validate(pipeline, X_sample, y_sample, cv=cv, scoring=scoring, return_train_score=True)

print(f"Acurácia - Treino: {cv_results['train_accuracy'].mean():.4f} ± {cv_results['train_accuracy'].std():.4f}")
print(f"Acurácia - Validação: {cv_results['test_accuracy'].mean():.4f} ± {cv_results['test_accuracy'].std():.4f}")
print(f"Precisão - Treino: {cv_results['train_precision'].mean():.4f} ± {cv_results['train_precision'].std():.4f}")
print(f"Precisão - Validação: {cv_results['test_precision'].mean():.4f} ± {cv_results['test_precision'].std():.4f}")
print(f"Recall - Treino: {cv_results['train_recall'].mean():.4f} ± {cv_results['train_recall'].std():.4f}")
print(f"Recall - Validação: {cv_results['test_recall'].mean():.4f} ± {cv_results['test_recall'].std():.4f}")
print(f"F1-score - Treino: {cv_results['train_f1'].mean():.4f} ± {cv_results['train_f1'].std():.4f}")
print(f"F1-score - Validação: {cv_results['test_f1'].mean():.4f} ± {cv_results['test_f1'].std():.4f}")

# Detectando overfitting
acc_diff = cv_results['train_accuracy'].mean() - cv_results['test_accuracy'].mean()
if acc_diff > 0.05:
    print(f"\n⚠️  POSSÍVEL OVERFITTING DETECTADO!")
    print(f"Diferença de acurácia treino-validação: {acc_diff:.4f}")
else:
    print(f"\n✅ Modelo parece estar generalizando bem.")
    print(f"Diferença de acurácia treino-validação: {acc_diff:.4f}")



VALIDAÇÃO CRUZADA 5-FOLDS
Acurácia - Treino: 0.9277 ± 0.0007
Acurácia - Validação: 0.9183 ± 0.0010
Precisão - Treino: 0.9972 ± 0.0001
Precisão - Validação: 0.9968 ± 0.0005
Recall - Treino: 0.8578 ± 0.0014
Recall - Validação: 0.8393 ± 0.0017
F1-score - Treino: 0.9223 ± 0.0008
F1-score - Validação: 0.9113 ± 0.0012

✅ Modelo parece estar generalizando bem.
Diferença de acurácia treino-validação: 0.0094


## Relatório de desempenho computacional

### Subtask:
Medir o tempo gasto no treinamento e na previsão e apresentar esses resultados.


**Reasoning**:
Measure the training time of the KNN model.



In [26]:
import time

start_time_train = time.time()
knn.fit(X_train_scaled, y_train)
end_time_train = time.time()

training_time = end_time_train - start_time_train
print(f"Training Time: {training_time:.4f} seconds")


Training Time: 0.1036 seconds


**Reasoning**:
Measure the prediction time of the KNN model and then display both training and prediction times.



In [27]:
pip install memory-profiler


[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [28]:
pip install psutil memory-profiler



[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [29]:
start_time_pred = time.time()
y_pred = knn.predict(X_test_scaled)
end_time_pred = time.time()

prediction_time = end_time_pred - start_time_pred
print(f"Prediction Time: {prediction_time:.4f} seconds")


Prediction Time: 13.0426 seconds


In [30]:
start_time_pred = time.time()
y_pred = knn.predict(X_test_scaled)
end_time_pred = time.time()

prediction_time = end_time_pred - start_time_pred
print(f"Prediction Time: {prediction_time:.4f} seconds")


Prediction Time: 11.4989 seconds


In [31]:
start_time_pred = time.time()
y_pred = knn.predict(X_test_scaled)
end_time_pred = time.time()

prediction_time = end_time_pred - start_time_pred
print(f"Prediction Time: {prediction_time:.4f} seconds")


Prediction Time: 11.4944 seconds


In [32]:
start_time_pred = time.time()
y_pred = knn.predict(X_test_scaled)
end_time_pred = time.time()

prediction_time = end_time_pred - start_time_pred
print(f"Prediction Time: {prediction_time:.4f} seconds")


Prediction Time: 11.3843 seconds


In [33]:
start_time_pred = time.time()
y_pred = knn.predict(X_test_scaled)
end_time_pred = time.time()

prediction_time = end_time_pred - start_time_pred
print(f"Prediction Time: {prediction_time:.4f} seconds")


Prediction Time: 12.6824 seconds


In [34]:
start_time_pred = time.time()
y_pred = knn.predict(X_test_scaled)
end_time_pred = time.time()

prediction_time = end_time_pred - start_time_pred
print(f"Prediction Time: {prediction_time:.4f} seconds")


Prediction Time: 10.2253 seconds


In [35]:
from sklearn.model_selection import cross_validate, StratifiedKFold
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score

knn_cv = KNeighborsClassifier(n_neighbors=15)
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

scoring = {
    'accuracy': make_scorer(accuracy_score),
    'precision': make_scorer(precision_score),
    'recall': make_scorer(recall_score),
    'f1': make_scorer(f1_score)
}

cv_results = cross_validate(knn_cv, X_sample, y_sample, cv=cv, scoring=scoring)

print("=== Validação Cruzada 5-Folds ===")
print(f"Acurácia média: {cv_results['test_accuracy'].mean():.4f}")
print(f"Precisão média: {cv_results['test_precision'].mean():.4f}")
print(f"Recall médio: {cv_results['test_recall'].mean():.4f}")
print(f"F1-score médio: {cv_results['test_f1'].mean():.4f}")


=== Validação Cruzada 5-Folds ===
Acurácia média: 0.7622
Precisão média: 0.9209
Recall médio: 0.5738
F1-score médio: 0.7070


In [36]:
import time
import psutil
from memory_profiler import memory_usage
from sklearn.neighbors import KNeighborsClassifier

process = psutil.Process()

# Definindo o classificador com k=15
knn = KNeighborsClassifier(n_neighbors=5)

# Função de treinamento com dados escalonados
def train_model():
    knn.fit(X_train_scaled, y_train)

start_time_train = time.time()
mem_usage_train = memory_usage(train_model)
end_time_train = time.time()
training_time = end_time_train - start_time_train
train_ips = len(X_train_scaled) / training_time

# Função de predição
def predict_model():
    global y_pred
    y_pred = knn.predict(X_test_scaled)

cpu_percent_before = process.cpu_percent(interval=None)
start_time_pred = time.time()
mem_usage_pred = memory_usage(predict_model)
end_time_pred = time.time()
cpu_percent_after = process.cpu_percent(interval=None)

prediction_time = end_time_pred - start_time_pred
pred_ips = len(X_test_scaled) / prediction_time

print("=== DESEMPENHO COMPUTACIONAL ===")
print(f"🕒 Tempo de Treinamento: {training_time:.4f} s")
print(f"🕒 Tempo de Predição: {prediction_time:.4f} s")
print(f"📈 Memória (Treinamento): {max(mem_usage_train):.2f} MB")
print(f"📈 Memória (Predição): {max(mem_usage_pred):.2f} MB")
print(f"⚙️ CPU usada na predição: {cpu_percent_after:.2f}%")
print(f"📊 Instâncias por segundo (treinamento): {train_ips:.2f}")
print(f"📊 Instâncias por segundo (predição): {pred_ips:.2f}")


=== DESEMPENHO COMPUTACIONAL ===
🕒 Tempo de Treinamento: 0.6636 s
🕒 Tempo de Predição: 11.3154 s
📈 Memória (Treinamento): 198.16 MB
📈 Memória (Predição): 201.12 MB
⚙️ CPU usada na predição: 91.20%
📊 Instâncias por segundo (treinamento): 105483.95
📊 Instâncias por segundo (predição): 2651.24


# 🔍 ANÁLISE DOS PROBLEMAS IDENTIFICADOS

## ❌ Problemas Críticos Corrigidos:

### 1. **VAZAMENTO DE DADOS (Data Leakage)**
- **Problema**: Predição no treino usando `X_train` (não escalonado) vs modelo treinado com `X_train_scaled`
- **Correção**: Usar `X_train_scaled` para predições no treino

### 2. **FEATURES COM VAZAMENTO TEMPORAL**
- **Problema**: Usar informações disponíveis apenas APÓS o voo (ARRIVAL_TIME, WHEELS_ON, etc.)
- **Correção**: Remover features que só estariam disponíveis após o evento que queremos prever

### 3. **FALTA DE VALIDAÇÃO CRUZADA**
- **Problema**: Não executar validação cruzada para detectar overfitting
- **Correção**: Implementar 5-fold cross-validation com pipeline adequado

## ✅ Melhorias Implementadas:

1. **Pipeline consistente** com StandardScaler + KNN
2. **Validação cruzada** para detectar overfitting
3. **Remoção de features problemáticas**
4. **Análise comparativa** treino vs validação

## 📊 Próximos Passos Recomendados:

1. **Re-executar o pipeline corrigido**
2. **Comparar métricas antes/depois das correções**
3. **Analisar se ainda há overfitting**
4. **Considerar feature engineering adicional**
5. **Testar outros valores de K no KNN**


## Summary:

### Data Analysis Key Findings

*   The data was split into training and testing sets with 80% for training and 20% for testing.
*   A K-Nearest Neighbors (KNN) classifier was trained on the training data.
*   The trained KNN model achieved an accuracy of approximately 93.11% on the test set.
*   The training time for the KNN model was approximately 0.0195 seconds.
*   The prediction time for the KNN model on the test set was approximately 14.4344 seconds.

### Insights or Next Steps

*   The KNN model shows a high accuracy on the test set, suggesting it performs well on this specific dataset.
*   The prediction time is significantly higher than the training time, which is characteristic of the KNN algorithm. For larger datasets or real-time predictions, exploring more computationally efficient models or optimization techniques for KNN might be necessary.


## Normalizar os dados

### Subtask:
Aplicar `StandardScaler` aos conjuntos de treino e teste para normalizar os dados.

**Reasoning**:
Split the balanced dataset into training and testing sets, train a KNN model, make predictions, and evaluate its performance using various metrics.



# Task
Perform hold-out validation (70/30 split with stratified sampling) and 5-fold cross-validation on the balanced dataset (`df_undersampled`) using a KNN model to evaluate its performance and assess for overfitting. Report the performance metrics (accuracy, precision, recall, F1-score, confusion matrix) for both validation strategies and compare the results.

# Task
Review the provided Python code and output from a machine learning experiment using KNN, focusing on data normalization, train/test split (70/30 stratified hold-out), k-fold cross-validation (k=5), and the analysis of performance metrics (accuracy, precision, recall, F1-score, TNR, confusion matrix) to identify potential issues, particularly overfitting, and suggest improvements.

## Revisão do código e lógica

### Subtask:
Analisar as células de código existentes para identificar possíveis erros, ineficiências ou inconsistências na lógica de processamento de dados e modelagem.


**Reasoning**:
Analyze the data loading and initial inspection steps.



**Reasoning**:
Analyze the data cleaning steps for potential errors or inefficiencies.



**Reasoning**:
Analyze the feature engineering and data balancing steps.



**Reasoning**:
Analyze the data splitting and scaling steps for correctness.

